In [10]:
# Backprop on the Vowel Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
import pandas as pd
import numpy as np
import csv

# Load a CSV file
def loadCsv(filename):
    trainSet = []

    with open(filename, 'r') as file:
        lines = csv.reader(file)
        dataset = list(lines)

        for i in range(len(dataset)):
            row = []
            for j in range(len(dataset[i])):
                try:
                    row.append(float(dataset[i][j]))
                except ValueError:
                    print("Non-numeric value at row {}, column {}: {}".format(i, j, dataset[i][j]))
                    continue
            trainSet.append(row)
    return trainSet

def minmax(dataset):
        minmax = list()
        stats = [[min(column), max(column)] for column in zip(*dataset)]
        return stats

# Rescale dataset columns to the range 0-1
def normalize(dataset, minmax):
        for row in dataset:
                for i in range(len(row)-1):
                        row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Convert string column to float
def column_to_float(dataset, column):
        for row in dataset:
                try:
                        row[column] = float(row[column])
                except ValueError:
                        print("Error with row",column,":",row[column])
                        pass

# Convert string column to integer
def column_to_int(dataset, column):
        class_values = [row[column] for row in dataset]
        unique = set(class_values)
        lookup = dict()
        for i, value in enumerate(unique):
                lookup[value] = i
        for row in dataset:
                row[column] = lookup[row[column]]
        return lookup

# Find the min and max values for each column


# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
        dataset_split = list()
        dataset_copy = list(dataset)
        fold_size = int(len(dataset) / n_folds)
        for i in range(n_folds):
                fold = list()
                while len(fold) < fold_size:
                        index = randrange(len(dataset_copy))
                        fold.append(dataset_copy.pop(index))
                dataset_split.append(fold)
        return dataset_split

# Calculate accuracy percentage
def accuracy_met(actual, predicted):
        correct = 0
        for i in range(len(actual)):
                if actual[i] == predicted[i]:
                        correct += 1
        return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def run_algorithm(dataset, algorithm, n_folds, *args):

        folds = cross_validation_split(dataset, n_folds)
        #for fold in folds:
                #print("Fold {} \n \n".format(fold))
        scores = list()
        for fold in folds:
                #print("Test Fold {} \n \n".format(fold))
                train_set = list(folds)
                train_set.remove(fold)
                train_set = sum(train_set, [])
                test_set = list()
                for row in fold:
                        row_copy = list(row)
                        test_set.append(row_copy)
                        row_copy[-1] = None
                predicted = algorithm(train_set, test_set, *args)
                actual = [row[-1] for row in fold]
                accuracy = accuracy_met(actual, predicted)
                cm = confusion_matrix(actual, predicted)
                print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in cm]))
                #confusionmatrix = np.matrix(cm)
                FP = cm.sum(axis=0) - np.diag(cm)
                FN = cm.sum(axis=1) - np.diag(cm)
                TP = np.diag(cm)
                TN = cm.sum() - (FP + FN + TP)
                print('False Positives\n {}'.format(FP))
                print('False Negetives\n {}'.format(FN))
                print('True Positives\n {}'.format(TP))
                print('True Negetives\n {}'.format(TN))
                TPR = TP/(TP+FN)
                print('Sensitivity \n {}'.format(TPR))
                TNR = TN/(TN+FP)
                print('Specificity \n {}'.format(TNR))
                Precision = TP/(TP+FP)
                print('Precision \n {}'.format(Precision))
                Recall = TP/(TP+FN)
                print('Recall \n {}'.format(Recall))
                Acc = (TP+TN)/(TP+TN+FP+FN)
                print('Áccuracy \n{}'.format(Acc))
                Fscore = 2*(Precision*Recall)/(Precision+Recall)
                print('FScore \n{}'.format(Fscore))
                k=cohen_kappa_score(actual, predicted)
                print('Çohen Kappa \n{}'.format(k))
                scores.append(accuracy)
        return scores

# Calculate neuron activation for an input
def activate(weights, inputs):
        activation = weights[-1]
        for i in range(len(weights)-1):
                activation += weights[i] * inputs[i]
        return activation

def transfer(activation, K):
    k0, k1 = K
    return k0 + k1 * activation

# Forward propagate input to a network output
def forward_propagate(network, row, K):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation, K)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output, K):
    return K[1]  # The derivative of k0 + k1 * x is k1


# Backpropagate error and store in neurons
def backward_propagate_error(network, expected, K):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'], K)

# Update network weights with error
def update_weights(network, row, l_rate, K, mu):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                temp = l_rate * neuron['delta'] * inputs[j] + mu * neuron['prev'][j]

                neuron['weights'][j] += temp
                neuron['prev'][j] = temp
            temp = l_rate * neuron['delta'] + mu * neuron['prev'][-1]
            neuron['weights'][-1] += temp
            neuron['prev'][-1] = temp



# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs, K):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row, K)  # Pass K to forward_propagate function
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected, K)  # Pass K to backward_propagate_error function
            update_weights(network, row, l_rate, K, mu)  # Pass K to update_weights function


# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
        network = list()
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]
        network.append(hidden_layer)
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]
        network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(n_hidden + 1)],'prev':[0 for i in range(n_hidden+1)]} for i in range(n_outputs)]
        network.append(output_layer)
        #print(network)
        return network

# Make a prediction with a network
def predict(network, row, K):
    outputs = forward_propagate(network, row, K)  # Pass K to forward_propagate function
    return outputs.index(max(outputs))

def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    K = (random(), random())  # Set initial random values for K
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs, K)  # Pass K to train_network function
    #print("network {}\n".format(network))
    predictions = list()
    for row in test:
        prediction = predict(network, row, K)  # Pass K to predict function
        predictions.append(prediction)
    return predictions


# Test Backprop on Seeds dataset
seed(1)
# load and prepare data
filename = 'banknote2.csv'
dataset = loadCsv(filename)
for i in range(len(dataset[0])-1):
        column_to_float(dataset, i)
# convert class column to integers
column_to_int(dataset, len(dataset[0])-1)
# normalize input variables
minmax = minmax(dataset)
normalize(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.1
mu=0.001
n_epoch = 1500
n_hidden = 4
scores = run_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)


 135   8
   0 131
False Positives
 [0 8]
False Negetives
 [8 0]
True Positives
 [135 131]
True Negetives
 [131 135]
Sensitivity 
 [0.94405594 1.        ]
Specificity 
 [1.         0.94405594]
Precision 
 [1.         0.94244604]
Recall 
 [0.94405594 1.        ]
Áccuracy 
[0.97080292 0.97080292]
FScore 
[0.97122302 0.97037037]
Çohen Kappa 
0.9416431499920133
 151   0
 123   0
False Positives
 [123   0]
False Negetives
 [  0 123]
True Positives
 [151   0]
True Negetives
 [  0 151]
Sensitivity 
 [1. 0.]
Specificity 
 [0. 1.]
Precision 
 [0.55109489        nan]
Recall 
 [1. 0.]
Áccuracy 
[0.55109489 0.55109489]
FScore 
[0.71058824        nan]
Çohen Kappa 
0.0


<ipython-input-10-2f7e6b8f1229>:122: RuntimeWarning: invalid value encountered in true_divide
  Precision = TP/(TP+FP)


 144   7
   0 123
False Positives
 [0 7]
False Negetives
 [7 0]
True Positives
 [144 123]
True Negetives
 [123 144]
Sensitivity 
 [0.95364238 1.        ]
Specificity 
 [1.         0.95364238]
Precision 
 [1.         0.94615385]
Recall 
 [0.95364238 1.        ]
Áccuracy 
[0.97445255 0.97445255]
FScore 
[0.97627119 0.97233202]
Çohen Kappa 
0.9486369235713138
 152   5
   0 117
False Positives
 [0 5]
False Negetives
 [5 0]
True Positives
 [152 117]
True Negetives
 [117 152]
Sensitivity 
 [0.96815287 1.        ]
Specificity 
 [1.         0.96815287]
Precision 
 [1.         0.95901639]
Recall 
 [0.96815287 1.        ]
Áccuracy 
[0.98175182 0.98175182]
FScore 
[0.98381877 0.9790795 ]
Çohen Kappa 
0.9629108235421517
 150   9
   0 115
False Positives
 [0 9]
False Negetives
 [9 0]
True Positives
 [150 115]
True Negetives
 [115 150]
Sensitivity 
 [0.94339623 1.        ]
Specificity 
 [1.         0.94339623]
Precision 
 [1.         0.92741935]
Recall 
 [0.94339623 1.        ]
Áccuracy 
[0.96715328

In [15]:
import matplotlib.pyplot as plt

# Extract the losses list from the 'trained_network'
losses = [tup[1] for tup in scores]

# Plot the loss function vs. epochs
plt.plot(range(1, n_epoch + 1), losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss function vs. Epochs')
plt.show()

TypeError: ignored